In [ ]:
import json, os

db = "postgres"

system_view_path = f"../../../knowledge_collection/{db}/knob_info/system_view.json"
knobs = []
with open(system_view_path, "r") as file:
    system_view = json.load(file)
    for key, value in system_view.items():
        knobs.append(key)
    knobs.sort()

# muli-source knowledge and summary
suggestions = dict()
path = f"../../../knowledge_collection/{db}/knowledge_sources/"
for knob in knobs:
    suggestions[knob] = dict()
    for i in ["gpt", "web", "manual"]:
        file_path = os.path.join(path, i, f"{knob}.txt")
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                txt = file.read()
            suggestions[knob][i] = txt
        else:
            suggestions[knob][i] = None
    summary_path = os.path.join(f"../../../knowledge_collection/{db}/tuning_lake", f"{knob}.txt")
    if os.path.exists(summary_path):
        with open(summary_path, 'r') as file:
            txt = file.read()
            suggestions[knob]["summary"] = txt
    else:
        suggestions[knob]["summary"] = None
    
print(suggestions)
with open(f"../suggestion_{db}.json", 'w') as file:
    json.dump(suggestions, file, indent=4)

In [ ]:
# integrate structured knowledge
structured_path = f"../../../knowledge_collection/{db}/structured_knowledge"

structured_knowledge = dict()
for knob in knobs:
    structured_knowledge[knob] = dict()
    file_path = os.path.join(structured_path, "normal", f"{knob}.json")
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            txt = json.load(file)
            structured_knowledge[knob] = txt
    else:
        structured_knowledge[knob]["min_value"] = None
        structured_knowledge[knob]["max_value"] = None
        structured_knowledge[knob]["suggested_values"] = []

    file_path = os.path.join(structured_path, 'special', f"{knob}.json")
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            txt = json.load(file)
            print(txt)
            print(knob)
            structured_knowledge[knob]["special_knob"] = txt["special_knob"]
            if structured_knowledge[knob]["special_knob"]:
                structured_knowledge[knob]["special_value"] = txt["special_value"]
            else:
                structured_knowledge[knob]["special_value"] = None
    else:
        structured_knowledge[knob]["special_knob"] = None
        structured_knowledge[knob]["special_value"] = None

print(structured_knowledge)
with open(f"../structured_knowlegde_{db}.json", 'w') as file:
    json.dump(structured_knowledge, file, indent=4)

In [2]:
# convert suggested_values to string
import json,os

with open(f"/home/knob/GPTuner/src/demo/structured_knowlegde_{db}.json", 'r') as f:
    data = json.load(f)
    for knob, value in data.items():
        if value["suggested_values"] is None:
            value["suggested_values"] = []
            data[knob] = value

with open(f"/home/knob/GPTuner/src/demo/structured_knowlegde_{db}.json", 'w') as f:
    json.dump(data, f, indent=4)